In [1]:
import ROOT

# PART 1

# look at slide 86

# define the observable
nobs = ROOT.RooRealVar("nobs", "n. of observed events", 0, 10)  # observable -> just the range
                       

In [2]:
# create a poisson model with signal strength
    
    # true, unknown n. of sig events 
mu = ROOT.RooRealVar("mu", "signal strength", 1, 0, 5)
nsig = ROOT.RooRealVar("nsig", "NOMINAL n. of signal events", 2.64 ) # CONSTANT (see Table 3 -> 2.64)

    # true, unknown n. of bkg events
nbkg = ROOT.RooRealVar("nbkg", "true, uknown n. of bkg events", 1, 0, 10) # PARAMETER 

    # true, unknown n. of total events
nexp = ROOT.RooFormulaVar("nexp", "@0 * @1 + @2", ROOT.RooArgList(mu, nsig, nbkg))    #  [mu, nsig, nbkg] -> in c++ is RooArgList(mu, nsig, nbkg)
  
   # poisson pdf

pois = ROOT.RooPoisson("pois", "pois", nobs, nexp)  

In [3]:
# create a guassian pdf to constraint background
# we need 3 variable: observable, mean and sigma

# we only have an estimate for "nbkg" with some uncertainty 
# see Table 3
# 0.25 ± 0.05


   # Global Observable (the estimate of nbkg, i.e.  0.25)
   # we need this object is considered as a 'special' observable in order not to lose the 
   # frequentist interpretation of our analysis
   # think this estimate as the 'results' of an imaginary experiment whose outcome is
   # gaussian-distributed around the true unknown background (nbkg) with sigma = 0.05
   #   
   # as an observable we need to provide a range (required to tossing experiment in MonteCarlo study)

nbkg_estimate = ROOT.RooRealVar("nbkg_estimate", "global observable", 0.25, 0, 10) 

   # sigma is 0.05
nbkg_err = ROOT.RooRealVar("sigma", "sigma", 0.05)  # CONSTANT (see Table 3 -> 0.05)

   # mean ? is already defined: nbkg (true, unknown n. of bkg events)

gaus = ROOT.RooGaussian("gaus", "gaus", nbkg_estimate, nbkg, nbkg_err)


[#0] WARNING:InputArguments -- The parameter 'sigma' with range [-inf, inf] of the RooGaussian 'gaus' exceeds the safe range of (0, inf). Advise to limit its range.


In [4]:
# put together with RooProdPdf

model = ROOT.RooProdPdf("model", "1 channel numb. counting model with contraints", [pois, gaus])

In [5]:
# create the dataset with the observation of the experiment
# see Table 3

data = ROOT.RooDataSet("data", "", nobs)

nobs.setVal(5)
data.add(nobs)
